In [1]:
import os
import json
import requests
import base64
from datetime import date
import re
import time
from pprint import pprint
from openpyxl import load_workbook
import xlsxwriter
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyotp import TOTP
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from seleniumbase import Driver

In [2]:
brand = "EBC"
print(f"Automated fulfillment started {brand}")

Automated fulfillment started EBC


In [3]:
#Login Details

GORGIAS_URL = "https://mlperformance-1.gorgias.com"
GORGIAS_OD_USERNAME = 'amanina@mlperformance.co.uk'
GORGIAS_OD_API_KEY = 'a40765a261a5096b6c4f04d16b25bf655cb77edb6c397ee88b8267480caa54b8'

AUTHORIZATION = base64.b64encode(
    f"{GORGIAS_OD_USERNAME}:{GORGIAS_OD_API_KEY}".encode("utf-8")
).decode("utf-8")

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Basic {AUTHORIZATION}",
}

# print(HEADERS)

In [4]:
#Supplier details

supplier_list = {"EBC": {
        "supplier_id" : 153616543, # supplier id in po shopify
        "supplier_email" : ["pkginfo@ups.com"],
        "customer_ids":[175689442], # cutomer id in gorgias
        "subject_pattern": "UPS Shipping Notification"
    }
}

#, \\d+ dispatched

In [5]:
def generate_ticket_list(tag_id):
    subject_pattern = supplier_list["EBC"]["subject_pattern"]
    customer_ids = supplier_list["EBC"]["customer_ids"]
    ticket_id_list = []

    for customer_id in customer_ids:
        #supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}&order_by=created_datetime%3Adesc&page=1&per_page=100"
        supplier_url = f"{GORGIAS_URL}/api/tickets?customer_id={customer_id}"
        print(supplier_url)
        response = requests.get(supplier_url, headers=HEADERS).json()
        # pprint(response)
        # cursor = response["meta"]["next_cursor"]

        # supplier_url_next = f"{supplier_url}&cursor={cursor}"

        # for data in response["data"]:
        #     print(data["id"])
        #     print(data["subject"])

        
        #     # print(data["tags"])

        #     if subject_pattern in data["subject"]:
        #         print("yes")


        
        ticket_id_list.extend(
            [
                data["id"]
                for data in response["data"]
                if (
                    re.search(subject_pattern, data["subject"])
                    # and any(tag_id in all_tag.values() for all_tag in data["tags"])
                    # and data["status"] == "open"
                )
            ]
        )
    return ticket_id_list

ticket_id_list = generate_ticket_list('711016')
ticket_id_list

https://mlperformance-1.gorgias.com/api/tickets?customer_id=175689442


[164787579, 164259905, 164259750, 163896555, 163884501, 163884496, 163760165]